#Mengimpor Library

In [ ]:
# Model
import numpy as np
import tensorflow as tf
import random

# Kontekstualisasi dan Pekerjaan NLP
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Pendukung
import json
import pickle
import warnings
import curses

warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Mengecek versi Tensorflow
print(tf. __version__)

2.12.0


#Pengolahan Dataset

In [ ]:
print("Memproses dataset intents.....")
with open('./assets/intents.json') as json_data:
    intents = json.load(json_data)

Processing the Intents.....


In [ ]:
words = []
classes = []
documents = []
ignore_words = ['?']
print("Mengiterasi isi dataset untuk mengubahnya menjadi words, class, documents, dan ignore_words.......")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenisasi tiap kata dalam kalimat
        w = nltk.word_tokenize(pattern)
        # Menambah kata ke dalam array words
        words.extend(w)
        # Menambah dokumen ke dalam korpus (pelatihan/uji)
        documents.append((w, intent['tag']))
        # Menambah tag ke dalam array class
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

Looping through the Intents to Convert them to words, classes, documents and ignore_words.......


In [ ]:
print("Meng-stemming, menggunakan huruf kecil dan menghapus duplikat kata.......")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

Stemming, Lowering and Removing Duplicates.......
213 documents
33 classes ['ac_arti', 'ac_bocor', 'ac_freon', 'ac_kompresor', 'ac_kotor', 'ac_merek', 'ac_rusak', 'audio_problems', 'battery_issues', 'blue_screen', 'charging_issues', 'complaint', 'create_account', 'driver_errors', 'hard_drive_failure', 'hp_baterai', 'hp_bluetooth', 'hp_bootlop', 'hp_iccharger', 'hp_lcd', 'hp_overheat', 'hp_wifi', 'keyboard_not_working', 'overheating', 'penutup', 'salam', 'screen_flickering', 'screen_problems', 'slow_performance', 'startup_issues', 'usb_port_issues', 'virus_infection', 'wifi_issues']
213 unique stemmed words [',', 'ac', 'ad', 'adalah', 'admin', 'air', 'ak', 'aku', 'akun', 'ap', 'artiny', 'assalamualaik', 'ata', 'bag', 'bagaiman', 'bantuanny', 'banyak', 'baru', 'bat', 'batera', 'baterainy', 'bau', 'berair', 'berfungs', 'berkedip', 'berkedip-kedip', 'berkurang', 'bermasalah', 'bersih', 'bertah', 'bertambah', 'biru', 'bis', 'bisakah', 'blank', 'blu', 'bluetoo', 'bluetoothny', 'blutut', 'boc

In [ ]:
print("Membuat data untuk model.....")
training = []
output = []
print("Membuat list kosong untuk output.....")
output_empty = [0] * len(classes)

print("Membuat training set dan Bag of Words ....")
for doc in documents:
    # Inisialisasi Bag of Words
    bag = []
    # Daftar kata-kata yang ditokenisasi untuk pattern
    pattern_words = doc[0]
    # Stem setiap kata
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # Membuat array Bag of Words
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # Outputnya '0' untuk setiap tag dan '1' untuk tag saat ini
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

Creating the Data for our Model.....
Creating an List (Empty) for Output.....
Creating Traning Set, Bag of Words for our Model....


#Membuat Training dan Test Set

In [ ]:
print("Mengacak dan mengubah menjadi numpy array untuk pemrosesan yang lebih cepat......")
random.shuffle(training)
training = np.array(training)

print("Membuat list train dan test.....")
train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

print("Shape train_x:", np.shape(train_x))
print("Shape train_y:", np.shape(train_y))

Shuffling Randomly and Converting into Numpy Array for Faster Processing......
Creating Train and Test Lists.....


#Membuat Model Sequential

In [ ]:
from tensorflow.keras.initializers import TruncatedNormal

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(8, input_shape=(len(train_x[0]),), activation='relu', kernel_initializer=TruncatedNormal()))
model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax', kernel_initializer=TruncatedNormal()))

# Kompilasi dan summary model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 1712      
                                                                 
 dense_1 (Dense)             (None, 33)                297       
                                                                 
Total params: 2,009
Trainable params: 2,009
Non-trainable params: 0
_________________________________________________________________


In [ ]:
print("Melakukan training model.......")
# Ensure the input shape matches the number of input features

model.fit(train_x, train_y, batch_size=8, epochs=200)

Training the Model.......
Epoch 1/200
27/27 [==============================] - 1s 2ms/step - loss: 3.4943 - acc: 0.0610 
Epoch 2/200
27/27 [==============================] - 0s 2ms/step - loss: 3.4829 - acc: 0.1408
Epoch 3/200
27/27 [==============================] - 0s 2ms/step - loss: 3.4672 - acc: 0.1878
Epoch 4/200
27/27 [==============================] - 0s 2ms/step - loss: 3.4402 - acc: 0.2113
Epoch 5/200
27/27 [==============================] - 0s 2ms/step - loss: 3.3986 - acc: 0.2300
Epoch 6/200
27/27 [==============================] - 0s 2ms/step - loss: 3.3407 - acc: 0.2160
Epoch 7/200
27/27 [==============================] - 0s 2ms/step - loss: 3.2697 - acc: 0.2394
Epoch 8/200
27/27 [==============================] - 0s 2ms/step - loss: 3.1899 - acc: 0.1643
Epoch 9/200
27/27 [==============================] - 0s 2ms/step - loss: 3.1097 - acc: 0.1690
Epoch 10/200
27/27 [==============================] - 0s 2ms/step - loss: 3.0271 - acc: 0.1549
Epoch 11/200
27/27 [============

In [ ]:
# Menyimpan model dan pickle
print("Melakukan save model.......")
model.save('model.h5')

print("Melakukan save pickle..........")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Pickle is also Saved..........


In [ ]:
print("Melakukan load pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('assets/intents.json') as json_data:
    intents = json.load(json_data)

print("Melakukan load model......")
# Melakukan loading model
model = tf.keras.models.load_model('./model.h5')

Loading Pickle.....
Loading the Model......


#Preprocessing input data dari user

In [ ]:
def clean_up_sentence(sentence):
    # Melakukan tokenisasi dan stemming
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Me-return array Bag of Words: True atau False dan 0 atau 1 untuk setiap kata di Bag of Words yang ada di kalimat user
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag).reshape(1, -1)  # Reshape to (1, num_features)

#Probabilitas maksimum untuk dihapus dari hasil
ERROR_THRESHOLD = 0.25

def classify(sentence):
    # Melakukan prediksi menggunakan model
    results = model.predict([bow(sentence, words)])[0]
    # Menghapus hasil yang dibawah treshhold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Mengsortir model berdasarkan probabilitas tertinggi
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # Menyesuaikan tag yang cocok
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    # Mengacak response
                    return print(random.choice(i['responses']))

            results.pop(0)

ERROR_THRESHOLD = 0.25


#Testing Model

In [ ]:
while True:
    input_data = input("You- ")
    if input_data.lower() in ['exit', 'selamat tinggal']:
        break
    else:
        answer = response(input_data)

1/1 [==============================] - 0s 89ms/step
Halo, Selamat datang di JasaIn Chatbot!. Jika kamu mempunyai kendala, silahkan deskripsikan kendalamu ya!


#Konversi TFLite

In [ ]:
# Konversi .h5 model menjadi TFLite model
converter = tf.lite.TFLiteConverter.from_keras_model('model.h5')
tflite_model = converter.convert()

# Melakukan save TFLite model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.


"\nfrom flask import Flask, jsonify, request\n# Create a Flask web application\n\napp = Flask(__name__)\n\n@app.route('/predict', methods=['POST'])\ndef predict():\n    # Get the data from the POST request\n    data = request.get_json()\n\n    # Preprocess the data if necessary\n    # ...\n\n    # Perform inference using the TFLite model\n    interpreter = tf.lite.Interpreter(model_path='your_model.tflite')\n    interpreter.allocate_tensors()\n\n    # Get input and output details\n    input_details = interpreter.get_input_details()\n    output_details = interpreter.get_output_details()\n\n    # Set the input tensor\n    input_data = data  # Modify if preprocessing is required\n    interpreter.set_tensor(input_details[0]['index'], input_data)\n\n    # Run inference\n    interpreter.invoke()\n\n    # Get the output tensor\n    output_data = interpreter.get_tensor(output_details[0]['index'])\n\n    # Postprocess the prediction if necessary\n    # ...\n\n    # Return the prediction as a JS